In [ ]:
# Julia version code
"""
nonnegative linear regression 
"""

# Notes
# 1. Note we set min A_ij = 1. This can be done either by dividing A by min Aij or by just adding 1. 
# The first approach is kind of cheating because the scaling automatically makes the error small
# If min Aij is not 1, then init error can be LARGE 

#2. If all xj coordinates are updated in parallel, then comparable to scipy (update_v_parllel and update_X_paralle)

#3. If coordinatewise update, then scaling down by n is better than not scaling. 

#4. Our theoreitcal alg in experiment has occasional large jumps in error

#5. The init large error issue is fixed by running a full update step first

#6. x is very very very sparse

#7. TODO m << n case 

import Pkg; Pkg.add("Plots")
using LinearAlgebra, BenchmarkTools, Distributions, Plots

const MultivariateDistribution{S<:ValueSupport} = Distribution{Multivariate,S}

const DiscreteMultivariateDistribution   = Distribution{Multivariate, Discrete}
const ContinuousMultivariateDistribution = Distribution{Multivariate, Continuous}

function compute_scaling(A)
    scaling_vector = -1 ./sum(abs2.(A),dims=1)
    return scaling_vector
end

function init_all(epsilon, m, n, A)
    ktotal = Int(ceil(n/sqrt(epsilon))) #note that this is just an approx ktotal
    
    akm = 1/n 
    Akm = 1/n
    ak = 1/n^2
    Ak = (1+n)/n^2
    
    scaling_vector = compute_scaling(A) 
    
    randomseed=rand(Multinomial(1, ones(n)/n),1)
    jk = findall(vec(randomseed.==1))[1] #
    
    v = zeros(n) # vector with same length as x, used to obtain x
    v[jk]=1 
    xkm = zeros(n) 
    xkm[jk]=-1/scaling_vector[jk] #xkm = x_1
    
    ykm = A*xkm # ykm = y_1 
    xtildek = xkm # wildtilde{x_k}
    ybar = (n+1)*ykm # ybar = ybar_1
    
    return ybar, ykm, xtildek, ktotal, ak, akm, Ak, Akm, v, xkm, scaling_vector
end

function update_v(jk, A, ybar, v, ak) 
    v[jk]+= n*ak*(dot(A[:,jk],ybar) - 1) # ybar = ybar_{k-1}
    return v
end

function update_x(v, jk, xkm, scaling) 
    x = xkm
    x[jk]= min(max(scaling*v[jk], 0),-scaling) # n has appeared in v
    return x      
end

function update_y(n, ak, Ak, Akm, x, xkm, xtildek, A)
    xtildek = (Akm/Ak)*xtildek+(n*ak/Ak)*x-((n-1)*ak/Ak)*xkm
    y =  A*xtildek
    return xtildek, y
end

function update_ak_Ak(Ak, Akm, ak, akm, n) 
    akm = ak
    ak = min(n*ak/(n-1),sqrt(Ak)/(2*n))
    Akm = Ak
    Ak = Ak+ ak
    return Ak, Akm, ak, akm
end

function update_ybar(y, ykm, ak, akm)
    ybar = y + (akm/ak)*(y - ykm)
    return ybar 
end



function remove_col1(A,b)
    s=A'*b # n*1 
    B=A[:,vec(s.>0)] # m*b matrix where b is smaller than n
    s=s[vec(s.>0)] # s is b*1
    A=B./s'
    return A, B, s
end

   Updating registry at `C:\Users\95889\.julia\registries\General`
  Resolving package versions...
  Installed Xorg_libXext_jll ───────────── v1.3.4+4
  Installed FriBidi_jll ────────────────── v1.0.5+6
  Installed XSLT_jll ───────────────────── v1.1.33+4
  Installed Gettext_jll ────────────────── v0.20.1+7
  Installed GeometryBasics ─────────────── v0.4.1
  Installed Colors ─────────────────────── v0.12.8
  Installed Plots ──────────────────────── v1.24.3
  Installed ColorSchemes ───────────────── v3.15.0
  Installed IterTools ──────────────────── v1.4.0
  Installed MacroTools ─────────────────── v0.5.9
  Installed Wayland_jll ────────────────── v1.17.0+4
  Installed GLFW_jll ───────────────────── v3.3.4+0
  Installed Showoff ────────────────────── v1.0.3
  Installed Xorg_xcb_util_image_jll ────── v0.4.0+1
  Installed DataValueInterfaces ────────── v1.0.0
  Installed URIs ───────────────────────── v1.3.0
  Installed TableTraits ────────────────── v1.0.1
  Installed Adapt ─────────────

  [0656b61e] + GLFW_jll v3.3.4+0
  [28b8d3ca] + GR v0.62.1
  [d2c73de3] + GR_jll v0.58.1+0
  [5c1252a2] + GeometryBasics v0.4.1
  [78b55507] + Gettext_jll v0.20.1+7
  [7746bdde] + Glib_jll v2.59.0+4
  [42e2da0e] + Grisu v1.0.2
  [cd3eb016] + HTTP v0.9.17
  [83e8ac13] + IniFile v0.5.0
  [c8e1da08] + IterTools v1.4.0
  [82899510] + IteratorInterfaceExtensions v1.0.0
  [aacddb02] + JpegTurbo_jll v2.0.1+3
  [c1c5ebd0] + LAME_jll v3.100.0+3
  [dd4b983a] + LZO_jll v2.10.0+3
  [b964fa9f] + LaTeXStrings v1.3.0
  [23fbe1c1] + Latexify v0.15.9
  [dd192d2f] + LibVPX_jll v1.9.0+1
  [e9f186c6] + Libffi_jll v3.2.1+4
  [d4300ac3] + Libgcrypt_jll v1.8.5+4
  [7e76a0d4] + Libglvnd_jll v1.3.0+3
  [7add5ba3] + Libgpg_error_jll v1.36.0+3
  [94ce4f54] + Libiconv_jll v1.16.0+8
  [4b2f31a3] + Libmount_jll v2.34.0+3
  [89763e89] + Libtiff_jll v4.1.0+2
  [38a345b3] + Libuuid_jll v2.34.0+7
  [1914dd2f] + MacroTools v0.5.9
  [442fdcdd] + Measures v0.3.1
  [77ba4419] + NaNMath v0.3.6
  [e7412a2a] + Ogg_jll v1.3.4+

In [ ]:
# Main loop
epsilon = 0.001 
n = 200 # input dimension 
m = 20 # Number of data 

# b can also be random and negative. m<<n.
b=rand(m,1)-repeat([0.3],m,1)
A=rand(m,n)

(A,B,s)=remove_col1(A,b)


(ybar, ykm, xtildek, ktotal, ak, akm, Ak, Akm, v, xkm, scaling_vector) = init_all(epsilon, m, n, A)
our_result = zeros(ktotal)    

for k in 2:ktotal 
        # sample jk from multinomial distribution
        randomseed=rand(Multinomial(1, ones(n)/n),1)
        jk = findall(vec(randomseed.==1))[1] # 
        
        # # update v 
        v = update_v(jk, A, ybar, v, ak) 
        
        # update x
        x = update_x(v, jk, xkm, scaling_vector[jk])
        
        # Update y based on u
        (xtildek, y) = update_y(n, ak, Ak, Akm, x, xkm, xtildek, A)
    
        #update a 
        (Ak, Akm, ak, akm) = update_ak_Ak(Ak, Akm, ak, akm, n)
    
        #update ybar 
        ybar = update_ybar(y, ykm, ak, akm)
     
        # update xkm 
        xkm = x 
        
        our_result[k] = norm(A*xtildek)^2/2-sum(xtildek)
        #our_result[k] = norm(A.dot(y),2)**2/2-sum(xsol_temp)
end

our_result1= xtildek
print(norm(A*our_result1)^2/2-sum(our_result))
f = plot(our_result)
@show f